## Dependencies



In [ ]:
!pip install python-dateutil

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Imports

In [ ]:
import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta
import plotly.express as px

## Load Fund Data and Pre-Process

In [ ]:
df_fund = pd.read_csv("../data/Parag Parikh Flexi Cap Fund - Direct Plan - Growth.csv", index_col="date")
df_fund.head()

,nav,dayChange
date,,
09-11-2022,51.7279,0.2760
07-11-2022,51.4519,0.4838
04-11-2022,50.9681,-0.3444
03-11-2022,51.3125,-0.3929
02-11-2022,51.7054,-0.2795


In [ ]:
df_fund.dtypes, df_fund.index.dtype

(nav          float64
 dayChange    float64
 dtype: object, dtype('O'))

In [ ]:
# pre-process
df_fund.index = pd.to_datetime(df_fund.index, format="%d-%m-%Y", utc=True)
df_fund["nav"] = df_fund["nav"].astype("float64")

# sort df_nav
df_fund = df_fund[::-1]

df_fund.index

DatetimeIndex(['2013-05-28 00:00:00+00:00', '2013-05-29 00:00:00+00:00',
               '2013-05-30 00:00:00+00:00', '2013-05-31 00:00:00+00:00',
               '2013-06-03 00:00:00+00:00', '2013-06-04 00:00:00+00:00',
               '2013-06-05 00:00:00+00:00', '2013-06-06 00:00:00+00:00',
               '2013-06-07 00:00:00+00:00', '2013-06-10 00:00:00+00:00',
               ...
               '2022-10-25 00:00:00+00:00', '2022-10-27 00:00:00+00:00',
               '2022-10-28 00:00:00+00:00', '2022-10-31 00:00:00+00:00',
               '2022-11-01 00:00:00+00:00', '2022-11-02 00:00:00+00:00',
               '2022-11-03 00:00:00+00:00', '2022-11-04 00:00:00+00:00',
               '2022-11-07 00:00:00+00:00', '2022-11-09 00:00:00+00:00'],
              dtype='datetime64[ns, UTC]', name='date', length=2325, freq=None)

In [ ]:
# data will always be missing for holidays, and weekends
# on holidays, the nav stays the same as previous day's nav

# refer: https://pandas.pydata.org/pandas-docs/stable/development/extending.html

@pd.api.extensions.register_dataframe_accessor("safe_nav")
class NAVAccessor:
    def __init__(self, pandas_obj):
        self._validate(pandas_obj)
        self.start_date = pandas_obj.index[0]
        self._obj = pandas_obj

    @staticmethod
    def _validate(obj):
        # verify there is a column nav,
        if "nav" not in obj.columns:
        # or not pd.api.types.is_datetime64_any_dtype(obj.index):
            raise AttributeError("Must have 'nav'")

    def is_date_available(self, date):
      return date in self._obj.index

    # let's create a separate function which handles this
    def for_date(self, date):
      if date > self.start_date:
        # go back by a day till holidays are not over 
        while not self.is_date_available(date):
          date -= datetime.timedelta(days=1)
        
        # return the data of the day before holidays
        return self._obj.loc[date]

      # return the data of inception if data is from very past
      return self._obj.loc[self.start_date]

In [ ]:
df_fund.safe_nav._obj.index

DatetimeIndex(['2013-05-28 00:00:00+00:00', '2013-05-29 00:00:00+00:00',
               '2013-05-30 00:00:00+00:00', '2013-05-31 00:00:00+00:00',
               '2013-06-03 00:00:00+00:00', '2013-06-04 00:00:00+00:00',
               '2013-06-05 00:00:00+00:00', '2013-06-06 00:00:00+00:00',
               '2013-06-07 00:00:00+00:00', '2013-06-10 00:00:00+00:00',
               ...
               '2022-10-25 00:00:00+00:00', '2022-10-27 00:00:00+00:00',
               '2022-10-28 00:00:00+00:00', '2022-10-31 00:00:00+00:00',
               '2022-11-01 00:00:00+00:00', '2022-11-02 00:00:00+00:00',
               '2022-11-03 00:00:00+00:00', '2022-11-04 00:00:00+00:00',
               '2022-11-07 00:00:00+00:00', '2022-11-09 00:00:00+00:00'],
              dtype='datetime64[ns, UTC]', name='date', length=2325, freq=None)

In [ ]:
df_fund.safe_nav._obj.index[0].day, df_fund.safe_nav._obj.index[0].month

(28, 5)

In [ ]:
# check if it works
today = datetime.datetime.now().replace(tzinfo=datetime.timezone.utc)
sunday_idx = (today.weekday() + 1) % 7

sun = today - datetime.timedelta(days=sunday_idx, hours=today.hour, minutes=today.minute, seconds=today.second, microseconds=today.microsecond)

sun = sun - datetime.timedelta(days=21)

print(sun)
print(df_fund.safe_nav.for_date(sun).name.day)

2022-11-06 00:00:00+00:00
4


In [ ]:
def get_date(year, month, day):
  return datetime.datetime(year=year, month=month, day=day, tzinfo=datetime.timezone.utc)

In [ ]:
def get_yesterday():
  today = datetime.datetime.now()
  yesterday = today - datetime.timedelta(days=1)
  yesterday = get_date(yesterday.year, yesterday.month, yesterday.day)
  return yesterday

## Calculate Returns

### Absolute Returns

In [ ]:
def absolute_return(initial_value, final_value):
  return (final_value - initial_value) * 100 / initial_value

In [ ]:
df_fund.safe_nav.for_date(get_yesterday())

nav          51.7279
dayChange     0.2760
Name: 2022-11-09 00:00:00+00:00, dtype: float64

In [ ]:
def one_day_absolute_return(df_nav):
  yesterday = get_yesterday()

  day_before_yesterday = yesterday - datetime.timedelta(days=1)

  return absolute_return(df_nav.safe_nav.for_date(day_before_yesterday)["nav"], df_nav.safe_nav.for_date(yesterday)["nav"])

one_day_absolute_return(df_fund)

0.0

In [ ]:
def one_year_absolute_return(df_nav):
  yesterday = get_yesterday()

  one_year_before_yesterday = yesterday - relativedelta(years=1)

  return absolute_return(df_nav.safe_nav.for_date(one_year_before_yesterday)["nav"], df_nav.safe_nav.for_date(yesterday)["nav"])

one_year_absolute_return(df_fund)

-3.2436122853615745

In [ ]:
df_fund.index[-1]

Timestamp('2022-11-09 00:00:00+0000', tz='UTC')

In [ ]:
def year_to_date_absolute_return(df_nav):
  yesterday = get_yesterday()
  
  year_start = datetime.datetime(year=datetime.datetime.now().year, month=1, day=1, tzinfo=datetime.timezone.utc)

  # handling holidays at the start of the year
  while not df_nav.safe_nav.is_date_available(year_start):
    year_start += datetime.timedelta(days=1)

  return absolute_return(df_nav.safe_nav.for_date(year_start)["nav"], df_nav.safe_nav.for_date(yesterday)["nav"])

year_to_date_absolute_return(df_fund)

-5.240124275487103

In [ ]:
# generic function

def n_years_absolute_return(df_nav, n_years=7):
  yesterday = get_yesterday()
  n_years_before_yesterday = (yesterday - relativedelta(years=n_years))

  return absolute_return(df_nav.safe_nav.for_date(n_years_before_yesterday)["nav"], df_nav.safe_nav.for_date(yesterday)["nav"])

In [ ]:
for n_years in (1, 3, 5, 7):
  return_ = n_years_absolute_return(df_fund, n_years=n_years)
  print("Years:", n_years, "\t", "Abs. Return:", return_)

Years: 1 	 Abs. Return: -3.2436122853615745
Years: 3 	 Abs. Return: 88.65094329300983
Years: 5 	 Abs. Return: 120.92722302895702
Years: 7 	 Abs. Return: 191.21806492292794


### Trailing Returns (AKA. CAGR i.e. Compounded Annual Growth Rate)

**Understanding**

Trailing Return cancels the compounding effect from the absolute return.

If it is a year-on-year compounding, trailing return tells you the yearly simple interest rate that would have applied to compound and reach the final amount in the given time span.

For example, the absolute return for 7 years, which is 196.37% does not consider the time that you'll have to wait to earn this much profit.

Like if we want to compare the return of this mutual fund with a typical compound interest scheme, we want to know the yearly simple interest rate that applies. What if a compound interest scheme for 7 years would generate more than 196.37% absolute return? You'd never know.

Trailing return helps us with calculating the yearly rate of interest that you'd need to reach upto 196.37% in 7 years. Basically, it remove the timefactor from the absolute return and gets you the yearly interest rate.

In [ ]:
# formulation

# compound interest formula

# final_value = initial_value * (1 + (interest_rate / num_times_interest_applied_per_period) ^ (num_times_interest_applied_per_period*num_periods))

# for 5 year compounding and interest applied yearly
# num_times_interest_applied_per_period = 1, num_periods=5

# for 5 year compounding and interest applied mothly
# num_times_interest_applied_per_period = 12, num_periods=5


# deriving the formula of trailing returns from above, we get

# trailing_return = ((final_value / initial_value) ^ (1 / num_times_interest_applied_per_period*num_periods) - 1) * num_times_interest_applied_per_period)

In [ ]:
# for most of the compounding schemes, num_times_interest_applied_per_period = 1

def trailing_return(initial_value, final_value, n_years):
  return (((final_value / initial_value) ** (1 / n_years)) - 1)*100

In [ ]:
def n_years_trailing_return(df_nav, n_years):
  yesterday = get_yesterday()
  n_years_before_yesterday = (yesterday - relativedelta(years=n_years))

  final_value = df_nav.safe_nav.for_date(yesterday)["nav"]
  initial_value = df_nav.safe_nav.for_date(n_years_before_yesterday)["nav"]

  return trailing_return(initial_value, final_value, n_years)

In [ ]:
for n_years in (1, 3, 5, 7):
  return_ = n_years_trailing_return(df_fund, n_years=n_years)
  print("Years:", n_years, "\t", "Trl. Return:", return_)

Years: 1 	 Trl. Return: -3.2436122853615745
Years: 3 	 Trl. Return: 23.56239768244537
Years: 5 	 Trl. Return: 17.179015998955236
Years: 7 	 Trl. Return: 16.497579411831097


**Output for future**

```
Years: 1 	 Abs. Return: -4.817920370508706
Years: 3 	 Abs. Return: 85.65517191136884
Years: 5 	 Abs. Return: 119.47250570555055
Years: 7 	 Abs. Return: 196.37096305255452
```

```
Years: 1 	 Trl. Return: -4.817920370508711
Years: 3 	 Trl. Return: 22.904848946865954
Years: 5 	 Trl. Return: 17.024292561361843
Years: 7 	 Trl. Return: 16.789847958013524
```

**Observations:**
1. The fund has performed significantly well over the past 3 years, however, it's performance dropped last year. Basically, the fund performed very well from November 2019 to November 2021, and it's performance dropped after November 2021 till November 2022.
2. We can confirm the same by looking at absolute returns. Out of 119.47% over 5 years, 85.65% alone was generate in the last 3 years.
3. The fund is not able to maintain it's yearly interest rate i.e. trailing return value in the last year. If you had to invest in this fund, ignoring the last year, you could expect an average annual return of 16% or more in this fund, given that you stay invested for more than 3 years.

**NOTE:** For 1 year, trailing return = absolute return as there is no compounding effect

### Rolling Returns

**Understanding**

Trailing Return gives you an estimate about the year-on-year performance of the fund, if invested for a long term.

Now, let's say for example that you have two funds having the same CAGR of 15% over the period of 5 years. How would you select one of them?

Or let's say we consider absolute returns, and both the funds have same return value of 100% over the period of 5 years. Again, how would you select a fund of these two?

What a professional would do is check for consistency.

For example, if fund A had +300% in the first 2 years, -100% in the 3rd year, +100% in the 4th year and -200% in the last year, and fund B had +50% in the first 2 years, -10% in the 3rd year, +20% in the 4th year and +40% in the fifth year, which one would you select?

I'd have opted for Fund B, as it is more stable and hence, less risky.

Rolling returns give you a way to check on the stability of returns by calculating the returns over a certain time period.

In the above case, we would select a 5 year time span and calculate different values of returns
- jan 2017 - jan 2018
- feb 2017 - feb 2018
- mar 2017 - mar 2018
- .
- .
- .
- dec 2020 - dec 2021
- jan 2021 - jan 2022


By having a look at these values, we will be able to understand how volatile/stable a fund is over the period of 5 years. The above values are calculated using 1 year data, hence they are called "rolling 1-year returns for a period of 5 years".

In [ ]:
df_fund

,nav,dayChange
date,,
2013-05-28 00:00:00+00:00,9.9992,NaN
2013-05-29 00:00:00+00:00,10.0080,0.0088
2013-05-30 00:00:00+00:00,10.0327,0.0247
2013-05-31 00:00:00+00:00,10.0154,-0.0173
2013-06-03 00:00:00+00:00,10.0572,0.0418
...,...,...
2022-11-02 00:00:00+00:00,51.7054,-0.2795
2022-11-03 00:00:00+00:00,51.3125,-0.3929
2022-11-04 00:00:00+00:00,50.9681,-0.3444


In [ ]:
# rolling returns over 5 year time span on a monthly basis
df_fund["nav"].rolling(365*5, closed="both").apply(lambda x: absolute_return(x.iloc[0], x.iloc[-1])).dropna()[::30]

date
2020-10-28 00:00:00+00:00    229.343347
2020-12-11 00:00:00+00:00    263.226753
2021-01-25 00:00:00+00:00    291.480866
2021-03-09 00:00:00+00:00    285.547260
2021-04-27 00:00:00+00:00    282.437033
2021-06-09 00:00:00+00:00    295.856433
2021-07-22 00:00:00+00:00    347.012985
2021-09-03 00:00:00+00:00    330.642390
2021-10-19 00:00:00+00:00    326.438323
2021-12-03 00:00:00+00:00    277.889376
2022-01-14 00:00:00+00:00    283.357544
2022-02-28 00:00:00+00:00    241.070993
2022-04-13 00:00:00+00:00    244.654482
2022-05-30 00:00:00+00:00    203.480677
2022-07-11 00:00:00+00:00    203.687845
2022-08-24 00:00:00+00:00    213.405936
2022-10-07 00:00:00+00:00    203.834941
Name: nav, dtype: float64

In [ ]:
# having a look at the above, it seems like the missing dates are causing issues

df_rr_temp = df_fund["nav"].rolling(365*5, closed="both").apply(lambda x: absolute_return(x.iloc[0], x.iloc[-1])).dropna()
df_rr_temp.groupby([df_rr_temp.index.year, df_rr_temp.index.month]).first()

date  date
2020  10      229.343347
      11      223.336689
      12      264.029197
2021  1       272.589714
      2       290.988640
      3       288.613027
      4       282.199661
      5       291.710946
      6       294.743545
      7       316.695727
      8       339.457820
      9       333.976606
      10      318.451892
      11      299.578808
      12      279.207291
2022  1       273.721648
      2       267.496191
      3       237.565468
      4       250.457457
      5       215.633134
      6       201.490579
      7       188.307505
      8       202.738981
      9       206.373022
      10      197.192762
      11      213.819936
Name: nav, dtype: float64

In [ ]:
df_rr_temp.groupby([df_rr_temp.index.year, df_rr_temp.index.month]).last()

date  date
2020  10      225.738409
      11      262.256205
      12      271.648662
2021  1       283.905749
      2       285.969350
      3       278.649799
      4       289.454252
      5       296.878502
      6       312.579515
      7       348.108853
      8       333.393185
      9       323.650311
      10      291.209099
      11      279.675418
      12      270.644806
2022  1       261.441171
      2       241.070993
      3       246.371659
      4       222.141278
      5       204.804605
      6       188.035676
      7       195.195596
      8       209.110783
      9       197.432963
      10      211.894855
      11      212.693212
Name: nav, dtype: float64

In [ ]:
# in reality, it must be daywise. so no need to group values 

def n_years_absolute_rolling_returns(df_nav, n_years):
  df_rr = df_nav["nav"].rolling(365*n_years, closed="both").apply(lambda window: absolute_return(window.iloc[0], window.iloc[-1])).dropna()
  return df_rr

In [ ]:
df_rr_temp = n_years_absolute_rolling_returns(df_fund, 5)
df_rr_temp.index

DatetimeIndex(['2020-10-28 00:00:00+00:00', '2020-10-29 00:00:00+00:00',
               '2020-10-30 00:00:00+00:00', '2020-11-02 00:00:00+00:00',
               '2020-11-03 00:00:00+00:00', '2020-11-04 00:00:00+00:00',
               '2020-11-05 00:00:00+00:00', '2020-11-06 00:00:00+00:00',
               '2020-11-09 00:00:00+00:00', '2020-11-10 00:00:00+00:00',
               ...
               '2022-10-25 00:00:00+00:00', '2022-10-27 00:00:00+00:00',
               '2022-10-28 00:00:00+00:00', '2022-10-31 00:00:00+00:00',
               '2022-11-01 00:00:00+00:00', '2022-11-02 00:00:00+00:00',
               '2022-11-03 00:00:00+00:00', '2022-11-04 00:00:00+00:00',
               '2022-11-07 00:00:00+00:00', '2022-11-09 00:00:00+00:00'],
              dtype='datetime64[ns, UTC]', name='date', length=501, freq=None)

In [ ]:
def n_years_trailing_rolling_returns(df_nav, n_years):
  df_rr = df_nav["nav"].rolling(365*n_years, closed="both").apply(lambda window: trailing_return(window.iloc[0], window.iloc[-1], n_years)).dropna()
  return df_rr

In [ ]:
n_years_trailing_rolling_returns(df_fund, 5)

date
2020-10-28 00:00:00+00:00    26.919917
2020-10-29 00:00:00+00:00    26.409720
2020-10-30 00:00:00+00:00    26.640843
2020-11-02 00:00:00+00:00    26.453542
2020-11-03 00:00:00+00:00    26.681263
                               ...    
2022-11-02 00:00:00+00:00    25.698006
2022-11-03 00:00:00+00:00    25.580864
2022-11-04 00:00:00+00:00    25.335028
2022-11-07 00:00:00+00:00    25.587622
2022-11-09 00:00:00+00:00    25.609848
Name: nav, Length: 501, dtype: float64

In [ ]:
df_fund.head()

,nav,dayChange
date,,
2013-05-28 00:00:00+00:00,9.9992,NaN
2013-05-29 00:00:00+00:00,10.0080,0.0088
2013-05-30 00:00:00+00:00,10.0327,0.0247
2013-05-31 00:00:00+00:00,10.0154,-0.0173
2013-06-03 00:00:00+00:00,10.0572,0.0418


In [ ]:
# the start year should be 2018, not 2020
# this happens because the window size does not account for holidays

def n_years_absolute_rolling_returns(df_nav, n_years, stride_timedelta=relativedelta(days=1)):
  rolling_returns = []
  dates = []

  start_date = df_nav.index[0]
  end_date = start_date + relativedelta(years=n_years)
  last_date = df_nav.index[-1]

  while end_date < last_date:
    df_window = df_nav.loc[start_date:end_date, "nav"]
    rolling_return = absolute_return(df_window.iloc[0], df_window.iloc[-1])

    rolling_returns.append(rolling_return)
    dates.append(end_date)

    start_date = start_date + stride_timedelta
    end_date = start_date + relativedelta(years=n_years)

  df_returns = pd.DataFrame(rolling_returns, columns=["rolling_returns"], index=dates)
  return df_returns

In [ ]:
df_abs_temp = n_years_absolute_rolling_returns(df_fund, 5)
df_abs_temp.index

DatetimeIndex(['2018-05-28 00:00:00+00:00', '2018-05-29 00:00:00+00:00',
               '2018-05-30 00:00:00+00:00', '2018-05-31 00:00:00+00:00',
               '2018-06-01 00:00:00+00:00', '2018-06-02 00:00:00+00:00',
               '2018-06-03 00:00:00+00:00', '2018-06-04 00:00:00+00:00',
               '2018-06-05 00:00:00+00:00', '2018-06-06 00:00:00+00:00',
               ...
               '2022-10-30 00:00:00+00:00', '2022-10-31 00:00:00+00:00',
               '2022-11-01 00:00:00+00:00', '2022-11-02 00:00:00+00:00',
               '2022-11-03 00:00:00+00:00', '2022-11-04 00:00:00+00:00',
               '2022-11-05 00:00:00+00:00', '2022-11-06 00:00:00+00:00',
               '2022-11-07 00:00:00+00:00', '2022-11-08 00:00:00+00:00'],
              dtype='datetime64[ns, UTC]', length=1626, freq=None)

In [ ]:
def n_years_trailing_rolling_returns(df_nav, n_years, stride_timedelta=relativedelta(days=1)):
  rolling_returns = []
  dates = []

  start_date = df_nav.index[0]
  end_date = start_date + relativedelta(years=n_years)
  last_date = df_nav.index[-1]

  while end_date < last_date:
    df_window = df_nav.loc[start_date:end_date, "nav"]
    rolling_return = trailing_return(df_window.iloc[0], df_window.iloc[-1], n_years)

    rolling_returns.append(rolling_return)
    dates.append(end_date)

    start_date = start_date + stride_timedelta
    end_date = start_date + relativedelta(years=n_years)

  df_returns = pd.DataFrame(rolling_returns, columns=["rolling_returns"], index=dates)
  return df_returns

In [ ]:
df_rr_five = n_years_trailing_rolling_returns(df_fund, 5)
df_rr_five

,rolling_returns
2018-05-28 00:00:00+00:00,19.449295
2018-05-29 00:00:00+00:00,19.407547
2018-05-30 00:00:00+00:00,19.348890
2018-05-31 00:00:00+00:00,19.388520
2018-06-01 00:00:00+00:00,19.262094
...,...
2022-11-04 00:00:00+00:00,16.997805
2022-11-05 00:00:00+00:00,16.997805
2022-11-06 00:00:00+00:00,16.997805
2022-11-07 00:00:00+00:00,17.297992


In [ ]:
df_rr_five.loc[get_date(2018, 9, 9)]

rolling_returns    21.440297
Name: 2018-09-09 00:00:00+00:00, dtype: float64

In [ ]:
df_rr_three = n_years_trailing_rolling_returns(df_fund, 3)
df_rr_three.loc[get_date(2018, 9, 9)]

rolling_returns    16.985412
Name: 2018-09-09 00:00:00+00:00, dtype: float64

In [ ]:
fig = px.line(df_rr_three, x=df_rr_three.index, y="rolling_returns", title="Rolling 3-year returns")
fig.show()

## Standard Deviation

Refer: https://scripbox.com/mf/standard-deviation-in-mutual-fund/

In [ ]:
df_rr_three.std()

rolling_returns    5.993386
dtype: float64

**Output for Future**
```
rolling_returns    5.993328
dtype: float64
```

The value shown on morningstar.in is `19.52`.

In [ ]:
df_rr_five.std()

rolling_returns    4.356336
dtype: float64

In [ ]:
def mean_and_standard_deviation(df_nav):
  # there won't be much change in return over one single day, but over a month, it would show a huge impact
  df_rr_for_std = n_years_absolute_rolling_returns(df_nav, n_years=1, stride_timedelta=relativedelta(months=1))
  # consider last 1-year (12 months) for standard deviation
  df_rr_for_std = df_rr_for_std.iloc[-12:]
  return df_rr_for_std.mean().item(), df_rr_for_std.std().item()

mean_and_standard_deviation(df_fund)

(19.015743614843128, 19.558409506889657)

**Output for future**
```
(19.015743614843128, 19.558409506889657)
```

**Understanding**

Here we are accounting for three factors:
- The variance in **yearly** returns, hence `n_years=1`
- Capture the change in returns every month (to allow some amount of variance), hence `stride=1 month`
- Consider only last 1 year data to calculate the standard deviation and mean, hence `iloc[-12:]`


The output says that the fund's yearly mean return is `19.0154%`, and it can go as down as `-0.54%` or upto `38.56%`.

This implies it's a highly risky scheme. Does it? We actually need to compare these values with the category average in order to decide how risky it is as compared to the funds in this category.

## Load TRI Data and Pre-process

In [ ]:
df_tri = pd.read_csv("../data/NIFTY 500_Data.csv", index_col="Date")
df_tri

,Open,High,Low,Close
Date,,,,
09 Nov 2022,15684.45,15686.35,15534.85,15567.25
07 Nov 2022,15610.95,15647.25,15505.75,15615.15
04 Nov 2022,15495.75,15542.15,15454.30,15530.85
03 Nov 2022,15407.80,15519.15,15401.30,15484.70
02 Nov 2022,15560.45,15561.1,15470.4,15497.55
...,...,...,...,...
06 Jan 2012,3682.05,3720.95,3651.1,3694.80
05 Jan 2012,3694.7,3718.75,3682.5,3696.10
04 Jan 2012,3706.05,3717.9,3681.75,3695.00


In [ ]:
df_tri.dtypes, df_tri.index.dtype

(Open      object
 High      object
 Low       object
 Close    float64
 dtype: object, dtype('O'))

In [ ]:
df_tri.index = pd.to_datetime(df_tri.index, utc=True)
df_tri.index

DatetimeIndex(['2022-11-09 00:00:00+00:00', '2022-11-07 00:00:00+00:00',
               '2022-11-04 00:00:00+00:00', '2022-11-03 00:00:00+00:00',
               '2022-11-02 00:00:00+00:00', '2022-11-01 00:00:00+00:00',
               '2022-10-31 00:00:00+00:00', '2022-10-28 00:00:00+00:00',
               '2022-10-27 00:00:00+00:00', '2022-10-25 00:00:00+00:00',
               ...
               '2012-01-12 00:00:00+00:00', '2012-01-11 00:00:00+00:00',
               '2012-01-10 00:00:00+00:00', '2012-01-09 00:00:00+00:00',
               '2012-01-07 00:00:00+00:00', '2012-01-06 00:00:00+00:00',
               '2012-01-05 00:00:00+00:00', '2012-01-04 00:00:00+00:00',
               '2012-01-03 00:00:00+00:00', '2012-01-02 00:00:00+00:00'],
              dtype='datetime64[ns, UTC]', name='Date', length=2690, freq=None)

In [ ]:
df_tri["nav"] = df_tri["Close"].astype("float64")

In [ ]:
df_tri = df_tri[::-1]

In [ ]:
df_tri.index.dtype, df_tri.dtypes

(datetime64[ns, UTC], Open      object
 High      object
 Low       object
 Close    float64
 nav      float64
 dtype: object)

In [ ]:
print(sun)
print(df_tri.safe_nav.for_date(sun))

2022-11-06 00:00:00+00:00
Open     15495.75
High     15542.15
Low      15454.30
Close    15530.85
nav      15530.85
Name: 2022-11-04 00:00:00+00:00, dtype: object


## Rolling Returns and Standard Deviation

In [ ]:
df_rr_three = n_years_trailing_rolling_returns(df_tri, 3)
fig = px.line(df_rr_three, x=df_rr_three.index, y="rolling_returns", title="Rolling 3-year returns")
fig.show()

In [ ]:
mean_and_standard_deviation(df_tri)

(13.696091261819006, 13.094633411762414)

## Alpha and Beta

References:
- https://scripbox.com/mf/alpha-and-beta-in-mutual-funds/
- https://www.morningstar.in/posts/63712/use-fund-risk-measure-tool.aspx
- https://www.wallstreetmojo.com/beta-formula/

**Risk Free Return**

10-yr GOI return rate: 7% (average and not so precise number)

The return changes over time

Rerence: http://www.worldgovernmentbonds.com/bond-historical-data/india/10-years/

In [ ]:
df_rr_three_fund = n_years_trailing_rolling_returns(df_fund, 3)
df_rr_three_tri = n_years_trailing_rolling_returns(df_tri, 3)
df_rr_three_fund.head(), df_rr_three_tri.head()

(                           rolling_returns
 2016-05-28 00:00:00+00:00        20.334470
 2016-05-29 00:00:00+00:00        20.299189
 2016-05-30 00:00:00+00:00        20.336137
 2016-05-31 00:00:00+00:00        20.328428
 2016-06-01 00:00:00+00:00        20.148637,
                            rolling_returns
 2015-01-02 00:00:00+00:00        23.986682
 2015-01-03 00:00:00+00:00        22.879045
 2015-01-04 00:00:00+00:00        22.943306
 2015-01-05 00:00:00+00:00        22.877375
 2015-01-06 00:00:00+00:00        21.681585)

In [ ]:
# let's join the dataframes by dates
df_rr_three_merged = pd.merge(df_rr_three_fund, df_rr_three_tri, how="inner", left_index=True, right_index=True, suffixes=("_fund", "_tri"))
df_rr_three_merged

,rolling_returns_fund,rolling_returns_tri
2016-05-28 00:00:00+00:00,20.334470,12.474484
2016-05-29 00:00:00+00:00,20.299189,12.556225
2016-05-30 00:00:00+00:00,20.336137,12.626439
2016-05-31 00:00:00+00:00,20.328428,13.273813
2016-06-01 00:00:00+00:00,20.148637,13.515951
...,...,...
2022-11-04 00:00:00+00:00,22.965048,16.857994
2022-11-05 00:00:00+00:00,22.904849,17.011889
2022-11-06 00:00:00+00:00,22.771132,16.902253
2022-11-07 00:00:00+00:00,23.205352,16.941416


In [ ]:
df_rr_three_merged.isna().any()

rolling_returns_fund    False
rolling_returns_tri     False
dtype: bool

In [65]:
# Beta = (Mutual Fund Return – Risk Free Rate (Rf­)) / (Benchmark Return – Risk Free Rate (Rf­))
risk_free_return = 7

df_beta = (df_rr_three_merged["rolling_returns_fund"] - risk_free_return) / (df_rr_three_merged["rolling_returns_tri"] - risk_free_return)
df_beta

2016-05-28 00:00:00+00:00    2.435749
2016-05-29 00:00:00+00:00    2.393566
2016-05-30 00:00:00+00:00    2.370262
2016-05-31 00:00:00+00:00    2.124454
2016-06-01 00:00:00+00:00    2.017915
                               ...   
2022-11-04 00:00:00+00:00    1.619503
2022-11-05 00:00:00+00:00    1.588596
2022-11-06 00:00:00+00:00    1.592681
2022-11-07 00:00:00+00:00    1.630085
2022-11-08 00:00:00+00:00    1.588273
Length: 2358, dtype: float64

In [66]:
# Beta = Covariance / Variance

df_rr_three_merged.cov()

,rolling_returns_fund,rolling_returns_tri
rolling_returns_fund,35.890653,26.079959
rolling_returns_tri,26.079959,28.507049


In [75]:
beta_all_time = df_rr_three_merged.cov().iloc[1,0] / df_rr_three_merged.cov().iloc[1,1]
beta_all_time

0.9148600002235181

In [72]:
def n_years_rolling_beta(df_rr_merged, n_years, stride_timedelta=relativedelta(days=1)):
  betas = []
  dates = []

  start_date = df_rr_merged.index[0]
  end_date = start_date + relativedelta(years=n_years)
  last_date = df_rr_merged.index[-1]

  while end_date < last_date:
    df_window = df_rr_merged.loc[start_date:end_date]
    beta = df_window.cov().iloc[1,0] / df_window.cov().iloc[1,1]

    betas.append(beta)
    dates.append(end_date)

    start_date = start_date + stride_timedelta
    end_date = start_date + relativedelta(years=n_years)

  df_beta = pd.DataFrame(betas, columns=["beta"], index=dates)
  return df_beta

In [78]:
df_beta = n_years_rolling_beta(df_rr_three_merged, n_years=1)
df_beta

,beta
2017-05-28 00:00:00+00:00,0.762851
2017-05-29 00:00:00+00:00,0.772355
2017-05-30 00:00:00+00:00,0.780656
2017-05-31 00:00:00+00:00,0.789358
2017-06-01 00:00:00+00:00,0.796426
...,...
2022-11-03 00:00:00+00:00,0.938637
2022-11-04 00:00:00+00:00,0.936218
2022-11-05 00:00:00+00:00,0.933636
2022-11-06 00:00:00+00:00,0.931164


In [80]:
fig = px.line(df_beta, x=df_beta.index, y="beta", title="Parag Parikh Flexi Cap Growth Direct - Beta calculated over 1-year period for 3-years return data")
fig.show()